In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import sys
import time
from sklearn.model_selection import train_test_split
from codes.GraphDPA import GCN_Model
from codes.utils import *
from prefetch_generator import BackgroundGenerator 
import torch.nn.functional as F

In [2]:
print('Start Time: {}'.format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))))

embed_dim = 11
num = 15
conv_type = 'GCNConv'
substructure_type = 'molecular graph'

if not os.path.exists('example_saved_data/model_tmp'):
    os.mkdir('example_saved_data/model_tmp')
if not os.path.exists('example_saved_data/results'):
    os.mkdir('example_saved_data/results')

with open('example_saved_data/entities2id.pkl'.format(substructure_type), 'rb') as file:
    entities2id = pickle.load(file) 

with open('example_saved_data/graphs/graph_map.pkl', 'rb') as file:
    graph_map = pickle.load(file)
    
res = {}
for i in range(num):
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    best_model_file = 'saved_models/{}+{}+{}+{}.pt'.format(substructure_type, embed_dim, i, conv_type)
    val_dataset = MyOwnDataset('example_saved_data', list(range(len(graph_map))), graph_map)
    val_loader = BackgroundGenerator(DataLoader(val_dataset, batch_size=1000))

    if not os.path.exists(best_model_file):
        print('ERROR-----The saved model file does not exist！！！！')

    model = torch.load(best_model_file, map_location=device)
    labels = []
    predicts = []

    for data in val_loader:
        data = data.to(device)
        logits = model(data)
        labels.extend(data.y.cpu().tolist())
        predicts.extend(F.softmax(logits, dim=1).cpu()[:,1].tolist())
    res['label'] = labels
    res[str(i)] = predicts
res = pd.DataFrame(res)
res['mean'] = res.iloc[:,1:].mean(axis=1)
res.to_csv('example_saved_data/results/{}_res.csv'.format(conv_type), index=False)

Start Time: 2021-10-18 01:09:47


In [3]:
res = pd.DataFrame(res)
res['mean'] = res.iloc[:,1:].mean(axis=1)
res.to_csv('example_saved_data/results/{}_res.csv'.format(conv_type), index=False)
print(evaluate(res['label'], res['mean'], 'None'))

{'Fold': 'None', 'ROC AUC': 0.7638842334593183, 'PR AUC': 0.7160436818743898}
